##### https://openmdao.org/newdocs/versions/latest/basic_user_guide/multidisciplinary_optimization/sellar.html

##### https://openmdao.org/newdocs/versions/latest/features/core_features/adding_desvars_cons_objs/main.html

In [4]:
import openmdao.api as om

# build the model
prob = om.Problem()

prob.model.add_subsystem('paraboloid', om.ExecComp('f = (x-3)**2 + x*y + (y+4)**2 - 3'))

# setup the optimization
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

prob.model.add_design_var('paraboloid.x', lower=-50, upper=50)
prob.model.add_design_var('paraboloid.y', lower=-50, upper=50)
prob.model.add_objective('paraboloid.f')

prob.setup()

# Set initial values.
prob.set_val('paraboloid.x', 3.0)
prob.set_val('paraboloid.y', -4.0)

# run the optimization
prob.run_driver();

Optimization terminated successfully    (Exit mode 0)
            Current function value: -27.33333333333333
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.
/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.
/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.


#### Computing y by solving for Eqn 1 [$cos(x \cdot y) - z \cdot y$]
#### Computing z after computing for y [ $sin(y)$ ]

##### x is `IndepVarComp`, y is `ImplicitComponent` and is computed with help of x and z, and z is `ExplicitComponent` which is computed with the help of computed y

In [5]:
import openmdao.api as om


class Paraboloid(om.ExplicitComponent):
    #Access ExplicitComponent class from openmdao.api.
    #This ExplicitComponent has 3 methods: setup, setup_partials, compute.
    
    """
    This is an Explicit function
    
    Evaluates the equation f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3.
    """

    def setup(self):
        #Define inputs and outputs in setup method under ExplicitComponent class.
        self.add_input('x', val=0.0)
        self.add_input('y', val=0.0)

        self.add_output('f_xy', val=0.0)

    def setup_partials(self): #What does this mean?
        #Apparently, this means asking OpenMDAO to approximate,
        #all partial (analytic) derivatives (derivative of O/P w.r.t. I/P).
        #This makes it easier to optimize.
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3

        Minimum at: x = 6.6667; y = -7.3333
        """
        x = inputs['x']
        y = inputs['y']

        outputs['f_xy'] = (x - 3.0)**2 + x * y + (y + 4.0)**2 - 3.0


if __name__ == "__main__":

    model = om.Group()
    #OpenMDAO models have a hierarchy of Group instances to organize components.
    #Here, there's only only single root group with single component/instance of defined Paraboloid class.
    model.add_subsystem('parab_comp', Paraboloid())

    prob = om.Problem(model)
    prob.setup()
    #To get some initial work to get data structures in place for execution.

    prob.set_val('parab_comp.x', 3.0)
    prob.set_val('parab_comp.y', -4.0)

    prob.run_model()
    #To perform computation
    print(prob['parab_comp.f_xy'])

    prob.set_val('parab_comp.x', 5.0)
    prob.set_val('parab_comp.y', -2.0)

    prob.run_model()
    #To perform computation
    print(prob.get_val('parab_comp.f_xy'))

[-15.]
[-5.]


/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.
/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.


##### We’ll add that component to construct our model inside a Problem. You’ve already used Problem in the run script from the previous tutorial on the paraboloid analysis, but we’ll take a closer look now.

##### All analyses and optimizations in OpenMDAO are executed with an instance of the Problem class. This class serves as a container for your model and the driver you’ve chosen, and provides methods for you to run the model and run the driver. It also provides a interface for setting and getting variable values. Every problem has a single driver associated with it; similarly, every problem has a single model in it.

In [6]:
# We'll use the component that was defined in the last tutorial
from openmdao.test_suite.components.paraboloid import Paraboloid

import openmdao.api as om

# build the model
prob = om.Problem()
prob.model.add_subsystem('parab', Paraboloid(), promotes_inputs=['x', 'y'])

# define the component whose output will be constrained
prob.model.add_subsystem('const', om.ExecComp('g = x + y'), promotes_inputs=['x', 'y'])

# Design variables 'x' and 'y' span components, so we need to provide a common initial
# value for them.
prob.model.set_input_defaults('x', 3.0)
prob.model.set_input_defaults('y', -4.0)

# setup the optimization
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'COBYLA'

prob.model.add_design_var('x', lower=-50, upper=50)
prob.model.add_design_var('y', lower=-50, upper=50)
prob.model.add_objective('parab.f_xy')

# to add the constraint to the model
prob.model.add_constraint('const.g', lower=0, upper=10.)

prob.setup()
prob.run_driver();

Optimization Complete
-----------------------------------

   Normal return from subroutine COBYLA

   NFVALS =   54   F =-2.700000E+01    MAXCV = 0.000000E+00
   X = 6.999999E+00  -6.999999E+00


/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.
/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.
/Users/h1he2li3/opt/anaconda3/envs/openmdao2/lib/python3.11/site-packages/openmdao/utils/reports_system.py:302: OpenMDAOWarning:Report with name 'inputs' not found in reports registry.


In [7]:
# minimum value
print(prob.get_val('parab.f_xy'))

[-27.]


In [8]:
# location of the minimum
print(prob.get_val('x'))
print(prob.get_val('y'))

[6.99999912]
[-6.99999912]
